# Load the data

It has been vectorized. There are no null values.

In [29]:
import pandas as pd
import numpy as np
import re

In [114]:
counsel = pd.read_json('../data/counsel_vect_topic.json')
counsel

,level_0,index,topic,sentence
0,0,0,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,0,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,0,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
3,3,0,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,0,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
60673,60673,2128,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
60674,60674,2128,2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
60675,60675,2128,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
60676,60676,2128,2,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


# Create the Model

I'll first try a neural network with Tensorflow and Keras.

In [18]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import json

In [122]:
counsel = counsel.sample(frac=1)
X = counsel.sentence
y = counsel.topic

X_train = np.vstack(X[:50000])
X_test = np.vstack(X[50000:])

y_train = y[:50000]
y_test = y[50000:]

y_train

50986     5
24890     5
2686      5
36186     4
16327    15
         ..
20655     9
32769    12
39293     9
16959    15
50131     4
Name: topic, Length: 50000, dtype: int64

In [123]:
X[:10]

50986    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
24890    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2686     [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...
36186    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
16327    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...
8503     [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...
33460    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
29203    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
10539    [0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, ...
27317    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: sentence, dtype: object

In [124]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1000, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(1000, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(1000, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(19, activation = tf.nn.softmax))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train, y_train, epochs=30)

Epoch 1/30
1563/1563 [==============================] - 43s 27ms/step - loss: 1.8554 - accuracy: 0.4144
Epoch 2/30
1563/1563 [==============================] - 44s 28ms/step - loss: 1.1291 - accuracy: 0.6393
Epoch 3/30
1563/1563 [==============================] - 44s 28ms/step - loss: 0.7334 - accuracy: 0.7513
Epoch 4/30
1563/1563 [==============================] - 45s 29ms/step - loss: 0.5862 - accuracy: 0.7893
Epoch 5/30
1563/1563 [==============================] - 42s 27ms/step - loss: 0.5254 - accuracy: 0.8029
Epoch 6/30
1563/1563 [==============================] - 47s 30ms/step - loss: 0.5022 - accuracy: 0.8072
Epoch 7/30
1563/1563 [==============================] - 43s 28ms/step - loss: 0.4827 - accuracy: 0.8120
Epoch 8/30
1563/1563 [==============================] - 44s 28ms/step - loss: 0.4732 - accuracy: 0.8141
Epoch 9/30
1563/1563 [==============================] - 46s 29ms/step - loss: 0.4562 - accuracy: 0.81690s - loss: 0.4564 - ac
Epoch 10/30
1563/1563 [===================

In [125]:
val_loss, val_acc = model.evaluate(X_test, y_test)
print(val_loss, val_acc)

334/334 [==============================] - 2s 6ms/step - loss: 3.0647 - accuracy: 0.6579
3.0646700859069824 0.6578947305679321


In [126]:
predic = model.predict([X_test])
print(predic)
print(y_test)

Consider rewriting this model with the Functional API.
[[3.5996917e-12 6.4191091e-10 3.0224252e-01 ... 3.8840170e-07
  1.2906658e-08 6.1064727e-09]
 [0.0000000e+00 1.7311806e-18 1.2403767e-09 ... 1.9228437e-14
  5.7847283e-09 2.1962042e-17]
 [1.3417711e-31 1.2174251e-17 2.9031520e-07 ... 1.7225778e-10
  1.6266826e-09 7.3801201e-09]
 ...
 [0.0000000e+00 6.1276869e-21 2.4080149e-16 ... 4.0004159e-15
  1.3784714e-17 1.0054126e-22]
 [1.7350956e-08 6.5848147e-03 7.8094564e-02 ... 4.7275744e-02
  1.0129165e-02 7.0840768e-03]
 [4.9011257e-11 2.1601278e-05 2.5223391e-03 ... 3.1536038e-03
  3.1444035e-04 6.9346500e-04]]
23347    12
12436    12
56266    14
18559     4
47237     8
         ..
13298     5
8855     12
15696     7
47680     8
55544     7
Name: topic, Length: 10678, dtype: int64


In [ ]:
"""model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(100, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(19, activation = tf.nn.softmax))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train, y_train, epochs=3)
loss: 1.2306 - accuracy: 0.5953
loss: 1.5089 - accuracy: 0.5312"""

"""model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(500, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(500, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(500, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(19, activation = tf.nn.softmax))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train, y_train, epochs=3)
loss: 0.7954 - accuracy: 0.7324
loss: 1.3477 - accuracy: 0.6274"""

"""model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1000, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(600, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(300, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(19, activation = tf.nn.softmax))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5)
loss: 0.5214 - accuracy: 0.8041
loss: 1.5806 - accuracy: 0.6456"""

"""model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1000, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(1000, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(1000, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(19, activation = tf.nn.softmax))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train, y_train, epochs=3)
loss: 0.7452 - accuracy: 0.7487
loss: 1.3906 - accuracy: 0.6306"""